# CatBoost

In [44]:
#Catboost
# !pip  install catboost -q
!pip install ipywidgets ipython -q

In [8]:
# Import Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
# encoders + Imputers
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer, SimpleImputer

In [21]:
# dataset
df = sns.load_dataset('titanic')

In [22]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [23]:
df.isnull().sum().sort_values(ascending = False)

deck           688
age            177
embarked         2
embark_town      2
sex              0
pclass           0
survived         0
fare             0
parch            0
sibsp            0
class            0
adult_male       0
who              0
alive            0
alone            0
dtype: int64

In [28]:
# impute the NAN values in fair, age, embark and embark_town
imputer = KNNImputer(n_neighbors=5)
df['fare'] = imputer.fit_transform(df[['fare']])
df['age'] = imputer.fit_transform(df[['age']])
# use simple imputer for categories
s_impu = SimpleImputer(strategy='most_frequent')
df['embarked'] = s_impu.fit_transform(df[['embarked']]).ravel()
df['embark_town'] = s_impu.fit_transform(df[['embark_town']]).ravel()
df['deck'] = s_impu.fit_transform(df[['deck']]).ravel()

In [29]:
df.isnull().sum().sort_values()

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
deck           0
embark_town    0
alive          0
alone          0
dtype: int64

In [ ]:
# deck column impted 600+ values which will bise the data, so remove that column
df.drop('deck', axis = 1, inplace=True)

In [33]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone'],
      dtype='object')

In [34]:
# convert each category column to category
categ_col = df.select_dtypes(include= ['object', 'category']).columns 

In [39]:
# convert columns to category
categ_col
df[categ_col] = df[categ_col].astype('category')

In [40]:
df.dtypes

survived          int64
pclass            int64
sex            category
age             float64
sibsp             int64
parch             int64
fare            float64
embarked       category
class          category
who            category
adult_male         bool
embark_town    category
alive          category
alone              bool
dtype: object

In [41]:
# data splitting
X = df.drop('survived', axis = 1)
y = df['survived']
#train test split
# split data into train & test 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=20, random_state=42)

In [46]:
# initiate model
model = CatBoostClassifier(iterations=100,
                           learning_rate=0.1,
                           depth=3,
                           loss_function='Logloss',
                          eval_metric='Accuracy',
                          random_seed=42,
                          verbose=False)
# fit model
model.fit(X_train, y_train, plot=True, cat_features=categ_col.tolist())
# predict 
y_predict = model.predict(X_test)
#Evaluate
print("Accuracy Score", accuracy_score(y_test, y_predict))
print("Confusion_Matrix", confusion_matrix(y_test, y_predict))
print("Classification Report", classification_report(y_test, y_predict))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Accuracy Score 1.0
Confusion_Matrix [[10  0]
 [ 0 10]]
Classification Report               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

